In [2]:
import tensorflow_datasets as tfds
import tensorflow as tf

/Users/jonatan/Library/Python/3.10/lib/python/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
dataset, info = tfds.load("tf_flowers",
                          as_supervised=True,
                          with_info=True)

2024-02-13 13:53:53.282742: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /Users/jonatan/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [4]:
dataset_size = info.splits["train"].num_examples
class_names = info.features["label"].names
n_classes = info. features["label"].num_classes

In [5]:
test_set_raw, valid_set_raw, train_set_raw = tfds.load("tf_flowers",
                        split=["train[:10%]",
                               "train[10%:25%]",
                               "train[25%:]"],
                        as_supervised=True)

In [6]:
def preprocess(image,label):
    resized_image = tf.image.resize(image, [224,224])
    final_image = tf.keras.applications.xception.preprocess_input(resized_image)
    return final_image, label


In [7]:
batch_size = 32
train_set = train_set_raw.shuffle(1000)
train_set = train_set.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

In [8]:
base_model = tf.keras.applications.xception.Xception(weights="imagenet",
                                            include_top=False)


avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(n_classes, activation="softmax")(avg)


model = tf.keras.Model(inputs=base_model.input, outputs=output)

83683744/83683744 [==============================] - 16s 0us/step


In [9]:
for layer in base_model.layers:
  layer.trainable = False

In [12]:
optimizer = tf.keras.optimizers.Adam(lr=0.2)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])


history = model.fit(train_set,
                    epochs=5,
                    validation_data=valid_set)

Epoch 1/5
86/86 [==============================] - 86s 981ms/step - loss: 0.6726 - accuracy: 0.7773 - val_loss: 0.5078 - val_accuracy: 0.8330
Epoch 2/5
86/86 [==============================] - 84s 976ms/step - loss: 0.3780 - accuracy: 0.8739 - val_loss: 0.4228 - val_accuracy: 0.8530
Epoch 3/5
86/86 [==============================] - 85s 988ms/step - loss: 0.3087 - accuracy: 0.8997 - val_loss: 0.3933 - val_accuracy: 0.8657
Epoch 4/5
86/86 [==============================] - 83s 964ms/step - loss: 0.2633 - accuracy: 0.9161 - val_loss: 0.3729 - val_accuracy: 0.8820
Epoch 5/5
86/86 [==============================] - 82s 950ms/step - loss: 0.2330 - accuracy: 0.9302 - val_loss: 0.3819 - val_accuracy: 0.8838


In [13]:
for layer in base_model.layers:
  layer.trainable = True

In [14]:
optimizer = tf.keras.optimizers.Adam(lr=0.01)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

history = model.fit(train_set,
                    epochs=10,
                    validation_data=valid_set)

Epoch 1/10
86/86 [==============================] - 3629s 43s/step - loss: 0.4876 - accuracy: 0.8332 - val_loss: 6.8004 - val_accuracy: 0.7241
Epoch 2/10
86/86 [==============================] - 4592s 54s/step - loss: 0.2787 - accuracy: 0.9113 - val_loss: 0.6686 - val_accuracy: 0.8530
Epoch 3/10
86/86 [==============================] - 351s 4s/step - loss: 0.1960 - accuracy: 0.9335 - val_loss: 0.6977 - val_accuracy: 0.8058
Epoch 4/10
86/86 [==============================] - 384s 4s/step - loss: 0.1210 - accuracy: 0.9608 - val_loss: 0.9773 - val_accuracy: 0.7659
Epoch 5/10
86/86 [==============================] - 429s 5s/step - loss: 0.1230 - accuracy: 0.9640 - val_loss: 0.5491 - val_accuracy: 0.8566
Epoch 6/10
23/86 [=======>......................] - ETA: 4:22 - loss: 0.0768 - accuracy: 0.9796

KeyboardInterrupt: 